# Por que métricas tradicionais de Machine Learning falham em modelos de decisão?

Este notebook demonstra, com um exemplo simples, por que métricas tradicionais de Machine Learning
(como acurácia) podem ser insuficientes quando o objetivo do modelo é **tomar decisões de negócio**,
e não apenas fazer boas previsões.


## Motivação

Em muitos problemas reais, Machine Learning não é usado apenas para prever resultados,
mas para **guiar decisões**.

Exemplos:
- Oferecer ou não um desconto
- Entrar ou não em contato com um cliente
- Aprovar ou não um crédito

O problema é que métricas tradicionais avaliam **o quão bem o modelo prevê o passado**,
mas não dizem se a **decisão sugerida realmente muda o resultado**.


## Modelos Preditivos vs Modelos de Decisão

🔹 **Modelo preditivo**  
Tenta responder:  
> "Qual é a probabilidade de um evento acontecer?"

Exemplo:
- Qual a probabilidade de um cliente cancelar?

🔹 **Modelo de decisão**  
Tenta responder:  
> "O que acontece se eu agir?"

Exemplo:
- O cliente cancelaria **mesmo se eu oferecer um desconto?**

Essa diferença é crucial — e muitas métricas de ML ignoram isso.


## Experimento Simulado

Neste experimento, vamos simular:
- Um problema de **churn**
- Uma ação de negócio: **oferecer desconto**
- Um modelo simples de regressão logística

O objetivo é mostrar que:
✔ O modelo pode ter alta acurácia  
❌ Mas ainda assim não responder à pergunta certa


In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [2]:
np.random.seed(42)

In [3]:
n = 1000

In [4]:
X = pd.DataFrame({
    "uso_produto": np.random.normal(50, 10, n),
    "reclamacoes": np.random.poisson(2, n),
    "desconto_oferecido": np.random.binomial(1, 0.5, n)
})

## Geração dos Dados

Criamos um conjunto de dados sintético com três variáveis:

- `uso_produto`: intensidade de uso do cliente
- `reclamacoes`: número de reclamações
- `desconto_oferecido`: ação controlável pelo negócio

Importante:
➡️ O desconto **realmente reduz churn no mundo simulado**  
➡️ Mas isso não fica claro apenas olhando para a acurácia


In [5]:
logit = (
    0.05 * X["uso_produto"]
    + 0.8 * X["reclamacoes"]
    - 1.5 * X["desconto_oferecido"]
)
prob_churn = 1 / (1 + np.exp(-logit))
y = np.random.binomial(1, prob_churn)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [7]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

## Treinamento do Modelo

Utilizamos uma **Regressão Logística**,
comumente usado para problemas de churn.

Aqui o modelo aprende **correlações** presentes nos dados,
mas não necessariamente **efeitos causais**.


In [8]:
y_pred = model.predict(X_test)
print("Acurácia:", accuracy_score(y_test, y_pred))

Acurácia: 0.9266666666666666


## Avaliação com Métrica Tradicional

Avaliamos o modelo usando **acurácia**.

Essa métrica responde bem à pergunta:
> "O modelo prevê corretamente quem vai cancelar?"

Mas **não responde**:
> "Oferecer desconto muda o comportamento do cliente?"
